In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('..')

In [2]:
from model.classification import *
from src import *

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

device = torch.device(f'cuda:1' if torch.cuda.is_available() else 'cpu')

In [4]:
files = [
    '20190819-Kutna Hora-L4-out-MVI_0040_manual',
    '20190819-Kutna Hora-L4-out-MVI_0040'
]

datapool = DataPool(files, 6, 0.75)

loading: 20190819-Kutna Hora-L4-out-MVI_0040_manual
 --> trn time 0 : 1542.0
 --> val time 1542.0 : 2064.0
loading: 20190819-Kutna Hora-L4-out-MVI_0040
 --> trn time 0 : 1542.0
 --> val time 1542.0 : 2064.0


In [48]:
manual_models = [
    1634898322,
    1634898797,
    1634899332
]

automatic_models = [
    1634896343,
    1634896834,
    1634897599
]

In [49]:
def get_error(signal, uuid, suffix, from_time, till_time, events):
    model, params = load_model(uuid, suffix)
    transform = create_transformation(params)
    params = get_additional_params(params)
    predictions = validate(signal, model, transform, params, from_time=from_time, till_time=till_time, classification=True)
    
    events = crop_events(events, from_time, till_time)
    n_events = len(events)
    error = np.abs(predictions.sum() - n_events)
    return error

In [50]:
file = '20190819-Kutna Hora-L4-out-MVI_0040_manual'

signal = load_audio(f'data/audio/{file}.MP4.wav')
events = load_events(f'data/labels/{file}.MP4.txt')

In [51]:
suffix = 'diff'

manual_errors = []
for uuid in manual_models:
    error = get_error(signal, uuid, suffix, 1542, 2064, events)
    print(f'{uuid}: {error:.4f}')
    manual_errors.append(error)
print(f'manual: {np.mean(manual_errors):.4f}\n')

automatic_errors = []
for uuid in automatic_models:
    error = get_error(signal, uuid, suffix, 1542, 2064, events)
    print(f'{uuid}: {error:.4f}')
    automatic_errors.append(error)
print(f'automatic: {np.mean(automatic_errors):.4f}\n')

1634898322: 0.0000
1634898797: 0.0000
1634899332: 1.0000
manual: 0.3333

1634896343: 1.0000
1634896834: 2.0000
1634897599: 1.0000
automatic: 1.3333

